# 04 Spark essentials

In [1]:
# Make it Python2 & Python3 compatible
from __future__ import print_function
import sys
if sys.version[0] == 3:
    xrange = range

## Spark context

The notebook deployment includes Spark automatically within each Python notebook kernel. This means that, upon kernel instantiation, there is an [SparkContext](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.SparkContext) object called `sc` immediatelly available in the Notebook, as in a PySpark shell. Let's take a look at it:

In [2]:
?sc

We can inspect some of the SparkContext properties:

In [3]:
# Spark version we are using
print( sc.version )

2.0.2


In [4]:
# Name of the application we are running
print( sc.appName )

PySparkShell


In [5]:
sc.appName

u'PySparkShell'

In [6]:
# Some configuration variables
print( sc.defaultParallelism )
print( sc.defaultMinPartitions )

2
2


In [7]:
# Username running all Spark processes
# --> Note this is a method, not a property
print( sc.sparkUser() )

vmuser


# Spark configuration

In [8]:
# Print out the SparkContext configuration
print( sc._conf.toDebugString() )

spark.app.id=local-1480797223440
spark.app.name=PySparkShell
spark.driver.host=10.0.2.15
spark.driver.port=59834
spark.eventLog.dir=/var/log/ipnb
spark.eventLog.enabled=true
spark.executor.id=driver
spark.files=file:/home/vmuser/.ivy2/jars/graphframes_graphframes-0.2.0-spark2.0-s_2.11.jar,file:/home/vmuser/.ivy2/jars/com.typesafe.scala-logging_scala-logging-api_2.11-2.1.2.jar,file:/home/vmuser/.ivy2/jars/com.typesafe.scala-logging_scala-logging-slf4j_2.11-2.1.2.jar,file:/home/vmuser/.ivy2/jars/org.scala-lang_scala-reflect-2.11.0.jar,file:/home/vmuser/.ivy2/jars/org.slf4j_slf4j-api-1.7.7.jar
spark.jars=file:/home/vmuser/.ivy2/jars/graphframes_graphframes-0.2.0-spark2.0-s_2.11.jar,file:/home/vmuser/.ivy2/jars/com.typesafe.scala-logging_scala-logging-api_2.11-2.1.2.jar,file:/home/vmuser/.ivy2/jars/com.typesafe.scala-logging_scala-logging-slf4j_2.11-2.1.2.jar,file:/home/vmuser/.ivy2/jars/org.scala-lang_scala-reflect-2.11.0.jar,file:/home/vmuser/.ivy2/jars/org.slf4j_slf4j-api-1.7.7.jar
spar

In [9]:
# Another way to get similar information
from pyspark import SparkConf, SparkContext
SparkConf().getAll()

[(u'spark.eventLog.enabled', u'true'),
 (u'spark.files',
  u'file:/home/vmuser/.ivy2/jars/graphframes_graphframes-0.2.0-spark2.0-s_2.11.jar,file:/home/vmuser/.ivy2/jars/com.typesafe.scala-logging_scala-logging-api_2.11-2.1.2.jar,file:/home/vmuser/.ivy2/jars/com.typesafe.scala-logging_scala-logging-slf4j_2.11-2.1.2.jar,file:/home/vmuser/.ivy2/jars/org.scala-lang_scala-reflect-2.11.0.jar,file:/home/vmuser/.ivy2/jars/org.slf4j_slf4j-api-1.7.7.jar'),
 (u'spark.jars.packages', u'graphframes:graphframes:0.2.0-spark2.0-s_2.11'),
 (u'spark.eventLog.dir', u'/var/log/ipnb'),
 (u'spark.master', u'local[*]'),
 (u'spark.jars',
  u'file:/home/vmuser/.ivy2/jars/graphframes_graphframes-0.2.0-spark2.0-s_2.11.jar,file:/home/vmuser/.ivy2/jars/com.typesafe.scala-logging_scala-logging-api_2.11-2.1.2.jar,file:/home/vmuser/.ivy2/jars/com.typesafe.scala-logging_scala-logging-slf4j_2.11-2.1.2.jar,file:/home/vmuser/.ivy2/jars/org.scala-lang_scala-reflect-2.11.0.jar,file:/home/vmuser/.ivy2/jars/org.slf4j_slf4j-a

## Spark execution modes

We can also take a look at the Spark configuration this kernel is running under, by using the above configuration data:

In [10]:
print( sc._conf.toDebugString() )

spark.app.id=local-1480797223440
spark.app.name=PySparkShell
spark.driver.host=10.0.2.15
spark.driver.port=59834
spark.eventLog.dir=/var/log/ipnb
spark.eventLog.enabled=true
spark.executor.id=driver
spark.files=file:/home/vmuser/.ivy2/jars/graphframes_graphframes-0.2.0-spark2.0-s_2.11.jar,file:/home/vmuser/.ivy2/jars/com.typesafe.scala-logging_scala-logging-api_2.11-2.1.2.jar,file:/home/vmuser/.ivy2/jars/com.typesafe.scala-logging_scala-logging-slf4j_2.11-2.1.2.jar,file:/home/vmuser/.ivy2/jars/org.scala-lang_scala-reflect-2.11.0.jar,file:/home/vmuser/.ivy2/jars/org.slf4j_slf4j-api-1.7.7.jar
spark.jars=file:/home/vmuser/.ivy2/jars/graphframes_graphframes-0.2.0-spark2.0-s_2.11.jar,file:/home/vmuser/.ivy2/jars/com.typesafe.scala-logging_scala-logging-api_2.11-2.1.2.jar,file:/home/vmuser/.ivy2/jars/com.typesafe.scala-logging_scala-logging-slf4j_2.11-2.1.2.jar,file:/home/vmuser/.ivy2/jars/org.scala-lang_scala-reflect-2.11.0.jar,file:/home/vmuser/.ivy2/jars/org.slf4j_slf4j-api-1.7.7.jar
spar

... this includes the execution mode for Spark. The default mode is *local*, i.e. all Spark processes run locally in the launched Virtual Machine. This is fine for developing and testing with small datasets.

But to run Spark applications on bigger datasets, they must be executed in a remote cluster. This deployment comes with configuration modes for that, which require:
* network adjustments to make the VM "visible" from the cluster: the virtual machine must be started in _bridged_ mode (the default *Vagrantfile* already contains code for doingso, but it must be uncommented)
* configuring the addresses for the cluster. This is done within the VM by using the `spark-notebook` script, such as
      sudo service spark-notebook set-addr <master-ip> <namenode-ip> <historyserver-ip>
* activating the desired mode, by executing
      sudo service spark-notebook set-mode (local | standalone | yarn)

These operations can also be performed outside the VM by telling vagrant to relay them, e.g.

    vagrant ssh -c "sudo service spark-notebook set-mode local"

## A trivial test

Let's do a trivial operation that creates an RDD and executes an action on it. So that we can test that the kernel is capable of launching executors

In [11]:
from operator import add

l = sc.parallelize( xrange(10000) )
print( l.reduce(add) )

49995000
